# Exploratory Notebook

In [1]:
import itertools
import numpy as np
import pandas as pd 
from numbers import Number
import sqlite3
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

## Assessing Box Office Mojo Data

In [2]:
bom_df = pd.read_csv('../zippedData/bom.movie_gross.csv.gz')

In [3]:
bom_df.head()

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010


In [4]:
bom_df.keys()

Index(['title', 'studio', 'domestic_gross', 'foreign_gross', 'year'], dtype='object')

In [5]:
bom_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   object 
 4   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB


In [6]:
bom_df['year'].unique()

array([2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018])

Looks like the data in this set is films released from 2010 - 2018

In [7]:
#Converting numeric foreign_gross values to floats, and all other values to NaN
bom_df['foreign_gross'] = pd.to_numeric(bom_df['foreign_gross'], errors='coerce')

#Finding the maximum and minimum values of the domestic & foreign columns
print(f"Max foreign gross: {bom_df['foreign_gross'].max()}")
print(f"Min foreign gross: {bom_df['foreign_gross'].min()}")
print()
print(f"Max domestic gross: {bom_df['domestic_gross'].max()}")
print(f"Min domestic gross: {bom_df['domestic_gross'].min()}")

Max foreign gross: 960500000.0
Min foreign gross: 600.0

Max domestic gross: 936700000.0
Min domestic gross: 100.0


In [8]:
bom_df['studio'].unique()

array(['BV', 'WB', 'P/DW', 'Sum.', 'Par.', 'Uni.', 'Fox', 'Wein.', 'Sony',
       'FoxS', 'SGem', 'WB (NL)', 'LGF', 'MBox', 'CL', 'W/Dim.', 'CBS',
       'Focus', 'MGM', 'Over.', 'Mira.', 'IFC', 'CJ', 'NM', 'SPC', 'ParV',
       'Gold.', 'JS', 'RAtt.', 'Magn.', 'Free', '3D', 'UTV', 'Rela.',
       'Zeit.', 'Anch.', 'PDA', 'Lorb.', 'App.', 'Drft.', 'Osci.', 'IW',
       'Rog.', nan, 'Eros', 'Relbig.', 'Viv.', 'Hann.', 'Strand', 'NGE',
       'Scre.', 'Kino', 'Abr.', 'CZ', 'ATO', 'First', 'GK', 'FInd.',
       'NFC', 'TFC', 'Pala.', 'Imag.', 'NAV', 'Arth.', 'CLS', 'Mont.',
       'Olive', 'CGld', 'FOAK', 'IVP', 'Yash', 'ICir', 'FM', 'Vita.',
       'WOW', 'Truly', 'Indic.', 'FD', 'Vari.', 'TriS', 'ORF', 'IM',
       'Elev.', 'Cohen', 'NeoC', 'Jan.', 'MNE', 'Trib.', 'Rocket',
       'OMNI/FSR', 'KKM', 'Argo.', 'SMod', 'Libre', 'FRun', 'WHE', 'P4',
       'KC', 'SD', 'AM', 'MPFT', 'Icar.', 'AGF', 'A23', 'Da.', 'NYer',
       'Rialto', 'DF', 'KL', 'ALP', 'LG/S', 'WGUSA', 'MPI', 'RTWC', 'FIP

In [9]:
bom_df['studio'].value_counts()

studio
IFC           166
Uni.          147
WB            140
Fox           136
Magn.         136
             ... 
E1              1
PI              1
ELS             1
PalT            1
Synergetic      1
Name: count, Length: 257, dtype: int64

In [10]:
#Locate the highest and lowest grossing films indexes
max_dom_index = bom_df['domestic_gross'].idxmax()
min_dom_index = bom_df['domestic_gross'].idxmin()
max_intl_index = bom_df['foreign_gross'].idxmax()
min_intl_index = bom_df['foreign_gross'].idxmin()

print("Studio with Maximum Domestic Gross:", bom_df.loc[max_dom_index, 'studio'])
print("Studio with Minimum Domestic Gross:", bom_df.loc[min_dom_index, 'studio'])
print()
print("Studio with Maximum Foreign Gross:", bom_df.loc[max_intl_index, 'studio'])
print("Studio with Minimum Foreign Gross:", bom_df.loc[min_intl_index, 'studio'])

Studio with Maximum Domestic Gross: BV
Studio with Minimum Domestic Gross: Magn.

Studio with Maximum Foreign Gross: WB
Studio with Minimum Foreign Gross: Fox


We know we have a lot of information on Magn, Fox, and WB. Let's check the value count for BV.

In [11]:
print((bom_df['studio'] == 'BV').sum())

106


In [12]:
#Cleaning the Magn cell value
bom_df['studio'] = bom_df['studio'].str.rstrip('.').str.strip()

In [13]:
#Create variables for studios with highest and lowest grossing films
bv_films = bom_df[bom_df['studio'] == 'BV']
magn_films = bom_df[bom_df['studio'] == 'Magn']
wb_films = bom_df[bom_df['studio'] == 'WB']
fox_films = bom_df[bom_df['studio'] == 'Fox']

#Find their unique value counts
bv_films.head()

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000.0,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000.0,2010
7,Tangled,BV,200800000.0,391000000.0,2010
13,Tron Legacy,BV,172100000.0,228000000.0,2010
15,Prince of Persia: The Sands of Time,BV,90800000.0,245600000.0,2010


### Keeping Box Office Mojo Data
This data has a ton of useful information. We'll briefly explore the other datasets, then return to this dataframe for further cleaning and analysis!

## Assessing Rotten Tomatoes Data

In [14]:
rt_movieinfo_df = pd.read_csv('../zippedData/rt.movie_info.tsv.gz', delimiter ='\t')
rt_movieinfo_df.head()

,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",NaN,NaN,104 minutes,NaN
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One
2,5,Illeana Douglas delivers a superb performance ...,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,"Sep 13, 1996","Apr 18, 2000",NaN,NaN,116 minutes,NaN
3,6,Michael Douglas runs afoul of a treacherous su...,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,"Dec 9, 1994","Aug 27, 1997",NaN,NaN,128 minutes,NaN
4,7,NaN,NR,Drama|Romance,Rodney Bennett,Giles Cooper,NaN,NaN,NaN,NaN,200 minutes,NaN


In [15]:
rt_movieinfo_df.keys()

Index(['id', 'synopsis', 'rating', 'genre', 'director', 'writer',
       'theater_date', 'dvd_date', 'currency', 'box_office', 'runtime',
       'studio'],
      dtype='object')

In [16]:
rt_movieinfo_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1560 entries, 0 to 1559
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            1560 non-null   int64 
 1   synopsis      1498 non-null   object
 2   rating        1557 non-null   object
 3   genre         1552 non-null   object
 4   director      1361 non-null   object
 5   writer        1111 non-null   object
 6   theater_date  1201 non-null   object
 7   dvd_date      1201 non-null   object
 8   currency      340 non-null    object
 9   box_office    340 non-null    object
 10  runtime       1530 non-null   object
 11  studio        494 non-null    object
dtypes: int64(1), object(11)
memory usage: 146.4+ KB


In [17]:
rt_reviews = pd.read_csv('../zippedData/rt.reviews.tsv.gz', delimiter='\t', encoding='latin1')

In [18]:
rt_reviews.head()

,id,review,rating,fresh,critic,top_critic,publisher,date
0,3,A distinctly gallows take on contemporary fina...,3/5,fresh,PJ Nabarro,0,Patrick Nabarro,"November 10, 2018"
1,3,It's an allegory in search of a meaning that n...,NaN,rotten,Annalee Newitz,0,io9.com,"May 23, 2018"
2,3,... life lived in a bubble in financial dealin...,NaN,fresh,Sean Axmaker,0,Stream on Demand,"January 4, 2018"
3,3,Continuing along a line introduced in last yea...,NaN,fresh,Daniel Kasman,0,MUBI,"November 16, 2017"
4,3,... a perverse twist on neorealism...,NaN,fresh,NaN,0,Cinema Scope,"October 12, 2017"


In [19]:
rt_reviews.keys()

Index(['id', 'review', 'rating', 'fresh', 'critic', 'top_critic', 'publisher',
       'date'],
      dtype='object')

In [20]:
rt_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54432 entries, 0 to 54431
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          54432 non-null  int64 
 1   review      48869 non-null  object
 2   rating      40915 non-null  object
 3   fresh       54432 non-null  object
 4   critic      51710 non-null  object
 5   top_critic  54432 non-null  int64 
 6   publisher   54123 non-null  object
 7   date        54432 non-null  object
dtypes: int64(2), object(6)
memory usage: 3.3+ MB


### Dropping Rotten Tomatoes Dataset from Analysis
Reviews are so subjective, and largely out of filmmaker/studio control. They are not a good metric by which to plan strategy for profit.

Furthermore, most of the values in 'box_office' for Rotten Tomatoes data is missing. Since we aren't given film titles, to cross reference with other dataframes, further exploring the RottenTomatoes data is not going to be the best use of our time.

## Assessing TheMovieDB Data

In [21]:
tmdb_df = pd.read_csv('../zippedData/tmdb.movies.csv.gz')
tmdb_df.head()

,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174
4,4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186


In [22]:
tmdb_df.keys()

Index(['Unnamed: 0', 'genre_ids', 'id', 'original_language', 'original_title',
       'popularity', 'release_date', 'title', 'vote_average', 'vote_count'],
      dtype='object')

In [23]:
tmdb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26517 entries, 0 to 26516
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         26517 non-null  int64  
 1   genre_ids          26517 non-null  object 
 2   id                 26517 non-null  int64  
 3   original_language  26517 non-null  object 
 4   original_title     26517 non-null  object 
 5   popularity         26517 non-null  float64
 6   release_date       26517 non-null  object 
 7   title              26517 non-null  object 
 8   vote_average       26517 non-null  float64
 9   vote_count         26517 non-null  int64  
dtypes: float64(2), int64(3), object(5)
memory usage: 2.0+ MB


### Setting Aside TheMovieDB Data

This data is also not particularly useful to our examination of monetary profit. The quantitative metrics of a films "popularity" could prove to be useful down the line, so for now we'll keep this dataset on the backburner.

## Assessing The Numbers Data

In [24]:
tn_budgets = pd.read_csv('../zippedData/tn.movie_budgets.csv.gz')
tn_budgets.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


In [25]:
tn_budgets.keys()

Index(['id', 'release_date', 'movie', 'production_budget', 'domestic_gross',
       'worldwide_gross'],
      dtype='object')

In [26]:
tn_budgets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   object
 4   domestic_gross     5782 non-null   object
 5   worldwide_gross    5782 non-null   object
dtypes: int64(1), object(5)
memory usage: 271.2+ KB


### Production Budget
Knowing a film's product budget would be incredibly useful. Then we can develop a metric of "net profit", comparing how much the film grossed after it's released compared to how much money was required to make the film. 

Let's convert all the cell values to integers, and see how many NaN values we actually have.

In [27]:
money_columns = ['production_budget', 'domestic_gross', 'worldwide_gross']

#Remove '$' signs from all cell values, before converting them to integers
for col in money_columns:
    tn_budgets[col] = tn_budgets[col].str.replace('$', '', regex=False)

### Cleaning cell values

When trying to convert the monetary columns into numeric datatypes, we've run into some issues.

Monetary values from other datasets do not contain commas. Lets remove commas as well to make it easier to cross reference between dataframes, and hopefully troubleshoot any formatting issues when converting between datatypes.

In [34]:
#Remove all commas
for col in money_columns:
    tn_budgets[col] = tn_budgets[col].str.replace(',','')

In [35]:
#Testing before modifying original column
prod_converted_values = pd.to_numeric(tn_budgets['production_budget'], errors='coerce')
prod_converted_values.head()

0    425000000
1    410600000
2    350000000
3    330600000
4    317000000
Name: production_budget, dtype: int64

In [36]:
prod_converted_values.info()

<class 'pandas.core.series.Series'>
RangeIndex: 5782 entries, 0 to 5781
Series name: production_budget
Non-Null Count  Dtype
--------------  -----
5782 non-null   int64
dtypes: int64(1)
memory usage: 45.3 KB


In [37]:
dom_converted_values = pd.to_numeric(tn_budgets['domestic_gross'], errors='coerce')
dom_converted_values.info()

<class 'pandas.core.series.Series'>
RangeIndex: 5782 entries, 0 to 5781
Series name: domestic_gross
Non-Null Count  Dtype
--------------  -----
5782 non-null   int64
dtypes: int64(1)
memory usage: 45.3 KB


In [38]:
intl_converted_values = pd.to_numeric(tn_budgets['worldwide_gross'], errors='coerce')
intl_converted_values.info()

<class 'pandas.core.series.Series'>
RangeIndex: 5782 entries, 0 to 5781
Series name: worldwide_gross
Non-Null Count  Dtype
--------------  -----
5782 non-null   int64
dtypes: int64(1)
memory usage: 45.3 KB


In [40]:
#Converting all original columns to numeric data types
for col in money_columns:
    tn_budgets[col] = pd.to_numeric(tn_budgets[col], errors='coerce')

tn_budgets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   int64 
 4   domestic_gross     5782 non-null   int64 
 5   worldwide_gross    5782 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 271.2+ KB


### DateTime, Isolating Year

In [42]:
tn_budgets['release_date'].value_counts()

release_date
Dec 31, 2014    24
Dec 31, 2015    23
Dec 31, 2010    15
Dec 31, 2008    14
Dec 31, 2009    13
                ..
Oct 21, 1977     1
Jun 23, 2009     1
Sep 26, 2017     1
Nov 14, 2007     1
Jul 13, 2005     1
Name: count, Length: 2418, dtype: int64

In [44]:
#Converting release_date to datetime values
tn_budgets['release_date'] = pd.to_datetime(tn_budgets['release_date'], format='%b %d, %Y')

#Creating new 'year' column with each films year
tn_budgets['year'] = tn_budgets['release_date'].dt.year

In [45]:
tn_budgets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   id                 5782 non-null   int64         
 1   release_date       5782 non-null   datetime64[ns]
 2   movie              5782 non-null   object        
 3   production_budget  5782 non-null   int64         
 4   domestic_gross     5782 non-null   int64         
 5   worldwide_gross    5782 non-null   int64         
 6   year               5782 non-null   int32         
dtypes: datetime64[ns](1), int32(1), int64(4), object(1)
memory usage: 293.7+ KB


In [47]:
tn_years = tn_budgets['year'].unique()
tn_years.sort()

tn_years

array([1915, 1916, 1920, 1925, 1927, 1929, 1930, 1931, 1933, 1934, 1935,
       1936, 1937, 1938, 1939, 1940, 1941, 1942, 1943, 1944, 1945, 1946,
       1947, 1948, 1949, 1950, 1951, 1952, 1953, 1954, 1955, 1956, 1957,
       1958, 1959, 1960, 1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968,
       1969, 1970, 1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979,
       1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990,
       1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001,
       2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012,
       2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020], dtype=int32)

### Drop outdated information
Our data includes films released from 1915-2020. Data from 1915 is certainly irrelevant. Let's drop everything prior to 2000, then look at our value counts to see if we narrow our search even further.

In [48]:
tn_budgets_filtered = tn_budgets[tn_budgets['year'] >= 2000]

In [52]:
tn_budgets_filtered['year'].value_counts().sort_index(ascending = False)

year
2020      3
2019     67
2018    143
2017    168
2016    219
2015    338
2014    255
2013    238
2012    235
2011    254
2010    274
2009    239
2008    264
2007    220
2006    260
2005    223
2004    206
2003    201
2002    210
2001    181
2000    189
Name: count, dtype: int64

For now, let's stick with films from the past 20 years. We can continue to filter as needed.

In [53]:
#Modifying original dataframe
tn_budgets = tn_budgets[tn_budgets['year'] >=2000]

### Keeping The Numbers Data
Similar to our Box Office Mojo dataset, this data also has a ton of useful information. We'll briefly explore the other datasets, then return to this dataframe for further cleaning and analysis!